Although [Fritz Zwicky](https://en.wikipedia.org/wiki/Fritz_Zwicky), in 1933, already had indications that things did not add up, and he coined the term *dunkle Materie* (Dark Matter, in German, he is from Swiss origin), it took another 20+ years before the astronomical community realized he was right.  Zwicky's work was based on the motions of galaxies in clusters, but it was the work of Robert, Rubin, Bosma and many others that solidified that in galaxies themselves there was an equal problem

In [ ]:
m31_data = './m31-rubin.tab'

read this data (x,y,vobs,weight)
and plot just the (x,y) positions on the sky. Units are arcmin. Recall the diameter of the moon is about 30 arcmin.  M31 is large!


The figure has X along the major axis, and Y along the minor axis, but these are rotated along the sky,because the position angle of the galaxy is not 90 degrees.


In [ ]:
PA = 32.0           # 0 means north up, positive is counting east (counter clock wise)
INC = 77.0          # 90 is edge on, 0 is face on
Distance = 670.0    # kpc

Now produce the same plot rotated how it should look on the sky and compare with the figure we took from the published paper.

Now produce the rotation speed, assuming all measurements are from the same plane and are on circular orbits, as function of a deprojected X. This would be rotation curve on both the negative and positive X' axis.

Fold it over, and plot a single rotation curve as function of radius. Convert the radius to kpc from the original arcmin based on the distance.